# 01. Feature Engineering & Matrix Construction

**Objective:** Transform raw data into a numerical matrix $X$ suitable for linear algebra operations.

**Steps:**
1. Load processed data.
2. Handle missing values.
3. One-Hot Encoding.
4. Save Matrix $X$ and Target $y$.

In [10]:
import os

# 1. Volvemos a la ruta raíz de Colab para ver la carpeta "desde fuera"
%cd /content

# 2. Nombre de la carpeta de tu repositorio
carpeta = "Credit-Risk-Algebraic-ML"

# 3. Ejecutamos el comando de sistema 'rm -rf'
# rm = remove (borrar)
# -r = recursive (borrar carpetas y subcarpetas)
# -f = force (no preguntar confirmación)
print(f"🗑️ Eliminando la carpeta: {carpeta}...")
!rm -rf {carpeta}

# 4. Verificación
if not os.path.exists(carpeta):
    print("✨ ¡Listo! La carpeta ha sido eliminada.")
    print("Ahora puedes volver a correr el bloque de 'Git Clone' una sola vez.")
else:
    print("⚠️ Algo pasó, la carpeta sigue ahí.")

/content
🗑️ Eliminando la carpeta: Credit-Risk-Algebraic-ML...
✨ ¡Listo! La carpeta ha sido eliminada.
Ahora puedes volver a correr el bloque de 'Git Clone' una sola vez.


In [11]:
import pandas as pd
import numpy as np
import os
import shutil

# --- 1. SETUP DEL ENTORNO (Git Clone) ---
if not os.path.exists('Credit-Risk-Algebraic-ML'):
    !git clone https://github.com/adriangonz-afk/Credit-Risk-Algebraic-ML.git
os.chdir('Credit-Risk-Algebraic-ML')

print("🔄 Iniciando Protocolo de Ingeniería de Datos...")

# --- 2. LIMPIEZA Y PREPARACIÓN ---
if os.path.exists('data'):
    shutil.rmtree('data')
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

# --- 3. DESCARGA OFICIAL (UCI - Raw Data) ---
# Esta es la fuente que SÍ funciona
raw_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
raw_path = "data/raw/german.data"
!wget -q {raw_url} -O {raw_path}

# --- 4. PARSING Y LIMPIEZA ---
columns = [
    'checking_account', 'duration', 'credit_history', 'purpose',
    'credit_amount', 'savings_account', 'employment_since', 'installment_rate',
    'personal_status_sex', 'other_debtors', 'residence_since', 'property',
    'age', 'other_installment_plans', 'housing', 'existing_credits',
    'job', 'people_liable', 'telephone', 'foreign_worker', 'target'
]

df = pd.read_csv(raw_path, names=columns, sep=' ', index_col=False)

# Transformar Target (1->0, 2->1)
df['target'] = df['target'].map({1: 0, 2: 1})

# One-Hot Encoding
df_encoded = pd.get_dummies(df, drop_first=True)

# Separar X e y
X = df_encoded.drop('target', axis=1).astype(float)
y = df_encoded['target']

# --- 5. GUARDADO ---
np.save('data/processed/X_matrix.npy', X.values)
np.save('data/processed/y_vector.npy', y.values)
df_encoded.to_csv('data/processed/german_credit_clean.csv', index=False)

# Guardar nombres de columnas (importante para el Notebook 02)
import json
with open('data/processed/feature_names.json', 'w') as f:
    json.dump(list(X.columns), f)

print(f"✅ ¡LISTO! Matrices generadas. Shape X: {X.shape}")

Cloning into 'Credit-Risk-Algebraic-ML'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 40 (delta 16), reused 32 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 46.14 KiB | 4.61 MiB/s, done.
Resolving deltas: 100% (16/16), done.
🔄 Iniciando Protocolo de Ingeniería de Datos...
✅ ¡LISTO! Matrices generadas. Shape X: (1000, 48)


In [12]:
df

,checking_account,duration,credit_history,purpose,credit_amount,savings_account,employment_since,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,existing_credits,job,people_liable,telephone,foreign_worker,target
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,0
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,1
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,0
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,0
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,0
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,0
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,0
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,1


In [17]:
import os
import getpass

# --- 1. SEGURIDAD: INGRESO INVISIBLE ---
print("🔐 Por favor, pega tu nuevo token cuando aparezca la casilla:")
# Esto pedirá el token sin guardarlo en el código
token = getpass.getpass('Token de GitHub: ')

username = "adriangonz-afk"
repo_name = "Credit-Risk-Algebraic-ML"

# --- 2. ACTUALIZACIÓN DE CREDENCIALES ---
# Construimos la URL segura usando el token que acabas de ingresar
git_url = f"https://{token}@github.com/{username}/{repo_name}.git"

print("\n🔧 Configurando acceso seguro...")

# Actualizamos la URL del remoto 'origin' para usar el nuevo token
!git remote set-url origin {git_url}

# --- 3. SUBIDA (PUSH) ---
print("🚀 Subiendo cambios (ahora sin revelar el secreto)...")

try:
    # Intentamos subir. Si el notebook se sube, solo llevará la línea 'getpass'
    !git push origin main
    print("\n✅ ¡ÉXITO! Tu código está en GitHub y tu token sigue secreto.")
except Exception as e:
    print(f"\n❌ Algo falló: {e}")

🔐 Por favor, pega tu nuevo token cuando aparezca la casilla:
Token de GitHub: ··········

🔧 Configurando acceso seguro...
🚀 Subiendo cambios (ahora sin revelar el secreto)...
Everything up-to-date

✅ ¡ÉXITO! Tu código está en GitHub y tu token sigue secreto.
